<a href="https://colab.research.google.com/github/dharmendra098/debrid/blob/main/torrent_search_and_get_debrid_link_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
#1337x
!pip install 1337x
#piratesbay
!git clone https://github.com/gp2112/pyratebay.git
!pip install pyratebay/.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.5 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=4f198118710a1661f6688f83f08ffe50a9f09ace63dcf79ecec7c1374fb40c66
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Cloning into 'pyratebay'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 36 (delta 14), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (36/36), 8.27 KiB | 1.18 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./pyratebay
  Preparing metad

In [2]:
import pandas as pd
import requests
from py1337x import py1337x
import pyratebay
torrents = py1337x(proxy='1337x.to', cache='py1337xCache', cacheTime=500)
# Using 1337x.tw and saving the cache in sqlite database which expires after 500 seconds

In [3]:
api='257Z637RY5VU2XLAEWM2ICYXQHS6YG4C3I7PIGW6W6C7TMXSBBMQ'

In [4]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Functions

In [83]:
df_torrents = pd.DataFrame({})
df_cached = pd.DataFrame({})
df_cloud = pd.DataFrame({})
df_files = pd.DataFrame({})
links = []
download_links = []

In [84]:
def file_list(dic):
    if isinstance(dic, dict):
        files_in_source = (
            pd.DataFrame([item for sublist in [list(d.items()) for d in dic['rd']] for item in sublist], columns=['sno', 'dic'])
            .drop_duplicates(subset='sno', keep='first')
            .reset_index(drop=True)
            
        )
        df = pd.concat([files_in_source.apply(lambda row: row.dic['filename'], axis = 1)
                        ,files_in_source.apply(lambda row: row.dic['filesize'], axis = 1)], axis=1)
        return df.to_numpy()
    else:
        return []

In [85]:
def size(n):
    n=n/1000000000
    gb= "{:.2f}".format(n)
    mb= "{:.2f}".format(n*1000)
    if(n>=1):
        return f"{gb}GB"
    else:
        return f"{mb}MB"    

In [86]:
def search_1337x(query, type_ ='All'):
    global df_torrents
    # get torrent list by search
    try:
        if(type_=='All'):
            results_dic = torrents.search(query)
        else:
            results_dic = torrents.search(query, category=type_)

        df = pd.DataFrame(results_dic['items'])
    except:
        print('site not accessible')
        return None
    
    
    # add info hash columns
    try:
        df['infoHash'] = 'NA'
        df['magnet'] = 'NA'
        for row in df.itertuples():
            try:
                torrent_info = torrents.info(link=row.link)
                df['infoHash'].iloc[row.Index] = torrent_info['infoHash']
                df['magnet'].iloc[row.Index] = torrent_info['magnetLink']
            except:
                pass
        df = df[df['infoHash'] != 'NA'] 

    except:
        print('info hash not added')
        return df[['name', 'seeders', 'leechers', 'size', 'time', 'uploader']]
    df_torrents = df[['name', 'seeders', 'leechers', 'size', 'time', 'uploader', 'infoHash', 'magnet']]

    return df_torrents[['name', 'seeders', 'leechers', 'size', 'time', 'uploader']]

In [87]:
def search_tpb(query):
    global df_torrents
    torrents = pyratebay.search(query)
    list_search_results=[]
    for torrent in torrents:
        list_search_results.append({'name':torrent.name, 'seeders': torrent.seeders, 'leechers': torrent.leechers, 'size':torrent.size, 'time': torrent.added, 'num_files': torrent.num_files, 'infoHash': torrent.info_hash, 'magnet': torrent.magnet()})

    df_torrents = pd.DataFrame(list_search_results)
    df_torrents['time'] = pd.to_datetime(df_torrents['time'],unit='s')
    df_torrents['size']=df_torrents['size'].astype(int).apply(size)

    return df_torrents[['name', 'seeders', 'leechers', 'size', 'time', 'num_files']]

In [93]:
def filter_cached():
        global df_cached
        df = df_torrents.copy()
        #check cached
        hash_list = '/'.join(df.loc[:,'infoHash'].to_list())
        url = f"https://api.real-debrid.com/rest/1.0/torrents/instantAvailability/{hash_list}"
        response = requests.get(
            url
            ,headers={"Authorization": f"Bearer {api}"}
        )
        df['cache_info'] = df.apply(lambda row: file_list(response.json()[row.infoHash.lower()]), axis = 1)
        df['number_of_files'] = df.apply(lambda row: len(row.cache_info), axis = 1)
        df=df[df['number_of_files']>0].reset_index(drop=True)
        df_cached=df

        return df_cached[['name', 'seeders', 'leechers', 'size', 'time', 'number_of_files']]

In [89]:
def get_debrid_link(i):
    global df_torrents, df_cloud, df_files, links
    # check link in cloud
    try:
        print('Cloud scanning')
        response = requests.get(
            'https://api.real-debrid.com/rest/1.0/torrents'
            ,headers={"Authorization": f"Bearer {api}"}
        )
        df_cloud = pd.DataFrame(response.json())
        info_hash = df_cached.loc[i, 'infoHash'].lower()
        matching_indexes = df_cloud.index[df_cloud['hash'] == info_hash].to_list()
        if len(matching_indexes):
            print('Found!')
            id_ = df_cloud.loc[matching_indexes[0], 'id']
            url = f"https://api.real-debrid.com/rest/1.0/torrents/info/{id_}"
            response = requests.get(
                url
                ,headers={"Authorization": f"Bearer {api}"}
            )
            df_files= pd.DataFrame(response.json()['files']).drop(['id', 'selected'], axis =1).rename({'path':'filename'}, axis=1)
            df_files['size']=df_files['bytes'].apply(size)
            df_files.drop(['bytes'], axis=1, inplace=True)
            return df_files
        else:
            print('Pass')
    except:
        print('cloud checking failed')
        pass
            
            
    
    #add magnet to debrid
    try:
        magnet = df_cached.loc[i,'magnet']

        response = requests.post(
            'https://api.real-debrid.com/rest/1.0/torrents/addMagnet'
            ,{"magnet" : magnet}
            ,headers={"Authorization": f"Bearer {api}"}
        )
        if not response.ok:
            print("error adding magnet")
            return None
    except:
        print('error adding magnet')
        return None
    
    #start magnet link
    try:
        id_ = response.json()['id']
        url = f"https://api.real-debrid.com/rest/1.0/torrents/selectFiles/{id_}"
        response = requests.post(
            url
            ,{"files": "all"}
            ,headers={"Authorization": f"Bearer {api}"}
        )

        if not response.ok:
            print("error starting magnet")
            return None

    except:
        print('error starting magnet')
        return None


    #get cached magnet link
    try:
        url = f"https://api.real-debrid.com/rest/1.0/torrents/info/{id_}"
        response = requests.get(
            url
            ,headers={"Authorization": f"Bearer {api}"}
        )
        links = response.json()['links']
        if response.ok:
            print('magnet added')
        else:
            print('error getting files')
            return None
    except:
        print('error getting files')
        return None
    
    df_files= pd.DataFrame(response.json()['files']).drop(['id', 'selected'], axis =1).rename({'path':'filename'}, axis=1)
    df_files['size']=df_files['bytes'].apply(size)
    df_files.drop(['bytes'], axis=1, inplace=True)
    return df_files

In [90]:
def unrestrict(i=[-1]):
    global download_links
    if isinstance(i, int):
        i_=[]
        i_.append(i)
        i=i_
    
    result = []
    #get unrestricted link
    for j,link in enumerate(links):
        if j not in i and i!=[-1]:
            continue
        try:
            response = requests.post(
                'https://api.real-debrid.com/rest/1.0/unrestrict/link'
                ,{"link": link}
                ,headers={"Authorization": f"Bearer {api}"}
            )

                        
            if not response.ok:
                print('error unrestricting')
                return None

            response = response.json()
            result.append((response['filename'],response['download']))
            download_links.append(response['download'])


        except:
            print('error unrestricting')
            return None

    return result

# Run

In [101]:
search_tpb('gta v')

,name,seeders,leechers,size,time,num_files
0,GTA V - Grand Theft Auto V - Rjaa,526,179,63.39GB,2019-08-20 10:38:02,11
1,GTA San Andreas-HOODLUM Plus SAMP Multiplayer,388,12,4.23GB,2012-06-21 07:41:21,4
2,GTA San Andreas v1.0 [Build 459558] Repack Tea...,138,1,3.63GB,2020-02-17 08:47:23,5
3,[PC Game] Gta Vice City (Multi5) TNT Village,77,2,1.35GB,2015-05-05 22:58:39,1
4,GTA Grand Theft Auto V (PC),59,20,63.38GB,2016-04-30 15:23:09,1
...,...,...,...,...,...,...
95,GTA Grand Theft Auto V (Xbox360),0,2,17.48GB,2016-12-17 20:44:59,2
96,GTA V DLC UPGRADE 4 Perfect Rip GLOBE,0,1,3.31GB,2017-06-27 21:57:19,1
97,GTA V MASSIVE Rip build1032on1.39RLD1.36+35dlc...,0,0,29.94GB,2017-07-06 16:59:35,14
98,GTA berty City v1.0 Repack Team-LiL,0,1,600.59MB,2020-03-08 12:53:51,3


In [91]:
search_1337x('asur')

,name,seeders,leechers,size,time,uploader
0,Asur (2023) S02 720p 10bit JC WEBRip x265 HEVC...,1757,600,2.0 GB,12pm Jun. 1st,iMMoRTaL69
1,Asur 2020 S01 Hindi 720p WEBRip x264 AAC - LOK...,538,549,2.9 GB,Mar. 21st '20,Lokioddin
2,Asur Season 02 S02 Hindi (1080p JIO WEBRip x26...,375,454,7.4 GB,6am Jun. 2nd,Musafirboy
3,Asur (2023) Season 2 S02 (1080p JIO WEBRip x26...,321,372,7.7 GB,6am Jun. 2nd,IONICBOII
4,"Asuran (2019) 720p WEBRip x264 AAC Multi [Hin,...",299,65,1.6 GB,Mar. 1st '21,krishh1337
5,Asur - Welcome to Your Dark Side (2020) Hindi ...,290,111,3.1 GB,Apr. 2nd '20,moviesbb
6,Asur S01 E01-08 WebDL Hindi 1080p x264 AAC - T...,253,262,6.2 GB,Mar. 21st '20,mkvCinemas
7,Asur (2020) Voot Select Hindi (S01 Complete E0...,247,61,1.7 GB,Mar. 4th '20,krishh1337
8,Asuravadham (2018)[Tamil HQ Real HQ CAM- x264 ...,201,40,423.7 MB,Jun. 29th '18,anily
9,Asuravadham (2018)[Tamil HQ Real HQ CAM - x264...,195,22,712.5 MB,Jun. 29th '18,anily


In [102]:
filter_cached()

,name,seeders,leechers,size,time,number_of_files
0,GTA V - Grand Theft Auto V - Rjaa,526,179,63.39GB,2019-08-20 10:38:02,11
1,GTA San Andreas-HOODLUM Plus SAMP Multiplayer,388,12,4.23GB,2012-06-21 07:41:21,4
2,[PC Game] Gta Vice City (Multi5) TNT Village,77,2,1.35GB,2015-05-05 22:58:39,1
3,GTA-IV v1.0.8.0 Repack Team-LiL,56,4,13.84GB,2020-03-05 14:37:23,9
4,Grand Theft Auto (GTA) San Andreas v1.06 Android,15,0,1.79GB,2014-10-27 00:46:00,3
5,Grand Theft Auto (GTA): San Andreas (PS2),11,0,2.79GB,2010-01-17 12:57:15,3
6,GTA Grand Theft Auto V (PS3),8,3,17.33GB,2016-04-22 20:27:55,4
7,PS2 - GTA San Andreas [1 DVD - PAL - MultiLang...,7,2,2.79GB,2010-04-27 00:20:50,29
8,PC Â» GTA IV PERFECT RIP directplay by globe@,4,0,4.76GB,2010-04-01 22:42:50,6
9,GTA Liberty City Stories [PSP],4,0,476.73MB,2013-12-16 04:15:36,1


In [103]:
get_debrid_link(0)

Cloud scanning
Pass
magnet added


,filename,size
0,/GamesTorrents - Descargar Juegos Torrent Grat...,0.00MB
1,/Importante.url,0.00MB
2,/Instrucciones.url,0.00MB
3,/Crack/steam_api.ini,0.00MB
4,/reloaded.nfo,0.01MB
5,/Crack/socialclub.ini,0.01MB
6,/Crack/doge64.dll,0.12MB
7,/Crack/steam_api_ext64.dll,0.34MB
8,/Crack/socialclub.dll,0.68MB
9,/Crack/steam_api64.dll,1.61MB


In [96]:
unrestrict()

[('Asur.S02E01.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-[Musafirboy].mkv',
  'https://54.download.real-debrid.com/d/6DX7NUTUAMLOS/Asur.S02E01.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv'),
 ('Asur.S02E02.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-[Musafirboy].mkv',
  'https://63.download.real-debrid.com/d/RTNUTFVR3Y2NA/Asur.S02E02.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv'),
 ('Asur.S02E03.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-[Musafirboy].mkv',
  'https://63.download.real-debrid.com/d/DTTCCTN53MNZS/Asur.S02E03.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv'),
 ('Asur.S02E04.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-[Musafirboy].mkv',
  'https://63.download.real-debrid.com/d/IJRGMAU4VLVTS/Asur.S02E04.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv'),
 ('Asur.S02E05.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-[Musafirboy].mkv',
  'https://63.download.real-debrid.com/d/W6EGE6RRK6T5E/Asur.S02E05.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv'),
 ('As

In [100]:
s=''
for link in download_links:
    s=s+link+'\n'
s

'https://54.download.real-debrid.com/d/6DX7NUTUAMLOS/Asur.S02E01.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/RTNUTFVR3Y2NA/Asur.S02E02.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/DTTCCTN53MNZS/Asur.S02E03.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/IJRGMAU4VLVTS/Asur.S02E04.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/W6EGE6RRK6T5E/Asur.S02E05.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/ATG6VAKXHTLAM/Asur.S02E06.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/6FHSYLE7R3YVA/Asur.S02E07.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\nhttps://63.download.real-debrid.com/d/2UHUPYQVEQIHY/Asur.S02E08.Hindi.1080p.WEBRip.10bit.DD.5.1.HEVC-%5BMusafirboy%5D.mkv\n'

# Stored data

In [40]:
df_torrents

,name,seeders,leechers,size,time,uploader,infoHash,magnet
0,Asur (2023) S02 720p 10bit JC WEBRip x265 HEVC...,1757,600,2.0 GB,12pm Jun. 1st,iMMoRTaL69,C6BFBC340ADE1ED85988DF574D1A1D4DF3ED9254,magnet:?xt=urn:btih:C6BFBC340ADE1ED85988DF574D...
1,Asur 2020 S01 Hindi 720p WEBRip x264 AAC - LOK...,538,549,2.9 GB,Mar. 21st '20,Lokioddin,87A9C8603CC40B01B5A2FAE9300B795D3B0EBEA5,magnet:?xt=urn:btih:87A9C8603CC40B01B5A2FAE930...
2,Asur Season 02 S02 Hindi (1080p JIO WEBRip x26...,375,454,7.4 GB,6am Jun. 2nd,Musafirboy,9316AB22FDD1D93ED5EB427352CE352431802C1E,magnet:?xt=urn:btih:9316AB22FDD1D93ED5EB427352...
3,Asur (2023) Season 2 S02 (1080p JIO WEBRip x26...,321,372,7.7 GB,6am Jun. 2nd,IONICBOII,82823623D03D2647C3FFEF8C3FC586E75E384880,magnet:?xt=urn:btih:82823623D03D2647C3FFEF8C3F...
4,"Asuran (2019) 720p WEBRip x264 AAC Multi [Hin,...",299,65,1.6 GB,Mar. 1st '21,krishh1337,D67F9ED1692F19D419CE8872B66F03188C221AEF,magnet:?xt=urn:btih:D67F9ED1692F19D419CE8872B6...
5,Asur - Welcome to Your Dark Side (2020) Hindi ...,290,111,3.1 GB,Apr. 2nd '20,moviesbb,828AD4CEFCB2D6344CFBD25ADD6FB96BD03C0010,magnet:?xt=urn:btih:828AD4CEFCB2D6344CFBD25ADD...
6,Asur S01 E01-08 WebDL Hindi 1080p x264 AAC - T...,253,262,6.2 GB,Mar. 21st '20,mkvCinemas,7816ACC02AAD8731FF29E147200D626C779E4FAC,magnet:?xt=urn:btih:7816ACC02AAD8731FF29E14720...
7,Asur (2020) Voot Select Hindi (S01 Complete E0...,247,61,1.7 GB,Mar. 4th '20,krishh1337,EC0A39B79910003E8CC8E3B2B63908CB6160582A,magnet:?xt=urn:btih:EC0A39B79910003E8CC8E3B2B6...
8,Asur S02 E01- 08 (2023) Hindi 1080p WEBRip x26...,153,91,5.6 GB,3am Jun. 2nd,krishh1337,195F600B8A8DA856ECEC637520663CBA59E88C0B,magnet:?xt=urn:btih:195F600B8A8DA856ECEC637520...
9,Asuravadham (2018)[Tamil HQ Real HQ CAM- x264 ...,201,40,423.7 MB,Jun. 29th '18,anily,154EF1D300382E1837E76E4FBB671832F4611BB7,magnet:?xt=urn:btih:154EF1D300382E1837E76E4FBB...


In [41]:
df_cached

,name,seeders,leechers,size,time,uploader,infoHash,magnet,cache_info,number_of_files
0,Asur (2023) S02 720p 10bit JC WEBRip x265 HEVC...,1757,600,2.0 GB,12pm Jun. 1st,iMMoRTaL69,C6BFBC340ADE1ED85988DF574D1A1D4DF3ED9254,magnet:?xt=urn:btih:C6BFBC340ADE1ED85988DF574D...,[[Asur S02E01 720p 10bit JC WEBRip x265 HEVC H...,8
1,Asur 2020 S01 Hindi 720p WEBRip x264 AAC - LOK...,538,549,2.9 GB,Mar. 21st '20,Lokioddin,87A9C8603CC40B01B5A2FAE9300B795D3B0EBEA5,magnet:?xt=urn:btih:87A9C8603CC40B01B5A2FAE930...,[[Asur 2020 S01E01 Hindi 720p WEBRip x264 AAC ...,10
2,Asur Season 02 S02 Hindi (1080p JIO WEBRip x26...,375,454,7.4 GB,6am Jun. 2nd,Musafirboy,9316AB22FDD1D93ED5EB427352CE352431802C1E,magnet:?xt=urn:btih:9316AB22FDD1D93ED5EB427352...,[[Asur.S02E01.Hindi.1080p.WEBRip.10bit.DD.5.1....,8
3,Asur (2023) Season 2 S02 (1080p JIO WEBRip x26...,321,372,7.7 GB,6am Jun. 2nd,IONICBOII,82823623D03D2647C3FFEF8C3FC586E75E384880,magnet:?xt=urn:btih:82823623D03D2647C3FFEF8C3F...,[[Asur.S02E08.1080p.10bit.WEBRip.DD.5.1x265.HE...,8
4,Asur - Welcome to Your Dark Side (2020) Hindi ...,290,111,3.1 GB,Apr. 2nd '20,moviesbb,828AD4CEFCB2D6344CFBD25ADD6FB96BD03C0010,magnet:?xt=urn:btih:828AD4CEFCB2D6344CFBD25ADD...,[[Asur - Welcome to Your Dark Side (2020) Hind...,1
5,Asur S01 E01-08 WebDL Hindi 1080p x264 AAC - T...,253,262,6.2 GB,Mar. 21st '20,mkvCinemas,7816ACC02AAD8731FF29E147200D626C779E4FAC,magnet:?xt=urn:btih:7816ACC02AAD8731FF29E14720...,"[[Asur Season01 Episode01.mp4, 895356067], [As...",8
6,Asur S02 E01- 08 (2023) Hindi 1080p WEBRip x26...,153,91,5.6 GB,3am Jun. 2nd,krishh1337,195F600B8A8DA856ECEC637520663CBA59E88C0B,magnet:?xt=urn:btih:195F600B8A8DA856ECEC637520...,"[[Asur S02 E08.mkv, 945464324], [Asur S02 E02....",8
7,Asur 2020 S01 Hindi 1080p WEBRip x264 AC3 - LO...,147,208,8.5 GB,Mar. 22nd '20,Lokioddin,0198859D70D73B330A867AB32FC72A9CDC95573D,magnet:?xt=urn:btih:0198859D70D73B330A867AB32F...,[[Asur 2020 S01E01 Hindi 1080p WEBRip x264 AC3...,9
8,Asur (2020) Bengali 720p Zee5 WEB-DL x264 AAC ...,115,17,1.1 GB,Apr. 10th '20,Suryadipta,7D142902AD2721D90578470C7F3E288E51B76327,magnet:?xt=urn:btih:7D142902AD2721D90578470C7F...,[[Asur (2020) Bengali 720p Zee5 WEB-DL x264 AA...,1


In [42]:
df_cloud

,id,filename,hash,bytes,host,split,progress,status,added,links,ended
0,AZIZEKKFS65MW,The.Hobbit.The.Desolation.of.Smaug.2013.Extend...,e070714262a673370f3198f8007c6710647caf23,4401034385,real-debrid.com,2000,100,downloaded,2023-06-03T19:44:21.000Z,[https://real-debrid.com/d/RJBX7URWPFYLY],2022-06-04T02:37:27.000Z
1,PFBRQSPQPJCRG,[ www.Torrent9.PH ] The.Equalizer.2.2018.MULTi...,083205cb568246f97959a85495bef5fb343211f5,4600054840,real-debrid.com,2000,100,downloaded,2023-06-03T19:00:01.000Z,[https://real-debrid.com/d/5642K5QOI2IPA],2022-10-01T21:24:27.000Z
2,MUENRLRAI3CDC,The Equalizer 2 (2018) (1080p BluRay x265 HEVC...,7831f94699725661c507c86fbd7994b2465b57ed,5612096627,real-debrid.com,2000,100,downloaded,2023-06-03T18:59:36.000Z,[https://real-debrid.com/d/MZJO42GPNWOJ6],2020-06-11T19:33:28.000Z
3,BMUBQEZGR4CU4,[ Torrent911.me ] To.Catch.a.Killer.2023.MULTi...,0952e2b163c000a70d161e553b45a5a3155fbd8d,5745210744,real-debrid.com,2000,100,downloaded,2023-06-03T06:20:10.000Z,[https://real-debrid.com/d/UQ246O6WDHCXG],2023-05-16T20:46:52.000Z
4,TR4XP7S7F4S4W,To.Catch.A.Killer.2023.1080p.WEB.H264-SLOT[rarbg],2abb6bd271ecbecebcc76e0ce578c977d06153af,6299712311,real-debrid.com,2000,100,downloaded,2023-06-03T06:19:56.000Z,[https://real-debrid.com/d/HHFQRP62U2PTC],2023-05-19T20:19:52.000Z
5,JT6NYZCM6D2IS,Asur S02 E01- 08 (2023) Hindi 1080p WEBRip x26...,195f600b8a8da856ecec637520663cba59e88c0b,6017961877,real-debrid.com,2000,100,downloaded,2023-06-02T21:25:40.000Z,"[https://real-debrid.com/d/WKIAUXS6PIYWE, http...",2023-06-02T07:58:10.000Z
6,L34Z26Z6YQK64,哈利·波特与火焰杯.Harry.Potter.and.the.Goblet.of.Fire....,d6bd32c2179d2c7bcf3c5e4becd7fc360ea53146,7981760626,real-debrid.com,2000,100,downloaded,2023-06-01T10:46:31.000Z,[https://real-debrid.com/d/BGWHSEQ5HOSES],2022-01-23T12:32:20.000Z
7,7F4E34AKBOMBG,Harry Potter and the Chamber of Secrets (2002)...,cdb1fbaf561a1b4ac49fb22c1382ae166946c1e6,6315661971,real-debrid.com,2000,100,downloaded,2023-06-01T07:06:49.000Z,[https://real-debrid.com/d/RLNXGVOK6YTMK],2021-09-05T08:46:41.000Z
8,UMTLWPICXMGQY,Harry Potter and the Philosopher's Stone (2001...,a3a36bb2fec71d8b8f70d527a0bdcf978ad9be69,5132726890,real-debrid.com,2000,100,downloaded,2023-06-01T06:56:52.000Z,[https://real-debrid.com/d/YXUDLIMEUZOTQ],2021-03-26T20:41:54.000Z
9,33PCPD6UEQCPU,Harry.Potter.And.The.Philosopher's.Stone.2001....,542dde48e0faa036f96542da32ab3c08c8a95698,5639269589,real-debrid.com,2000,100,downloaded,2023-06-01T06:56:47.000Z,[https://real-debrid.com/d/ZKVKZSQZOSEXQ],2022-10-05T21:40:18.000Z


In [43]:
df_files

,filename,size
0,/Asur - Welcome to Your Dark Side (2020) Hindi...,3.29GB


In [44]:
links

['https://real-debrid.com/d/QHFNTQUR546DG']